In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install pillow
!pip install easyocr
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
%%writefile app09v2.py

import streamlit as st
import joblib
import pandas as pd
import praw
import easyocr
from PIL import Image
from deep_translator import GoogleTranslator
import requests
from io import BytesIO
from collections import Counter

# Load the saved logistic regression model and vectorizer
model = joblib.load('LRmodel.pkl')
vectorizer = joblib.load('LRvectorizer.pkl')

# Initialize OCR reader and Reddit API client
reader = easyocr.Reader(['en', 'es', 'fr', 'de'])  # Add other languages if needed
reddit = praw.Reddit(client_id='<REDDIT_CLIENT_ID>',
                     client_secret='<REDDIT_CLIENT_SECRET>',
                     user_agent='Mental Health')

# Function to fetch images from Reddit user posts
def fetch_user_images(username):
    try:
        user = reddit.redditor(username)
        images = [post.url for post in user.submissions.new(limit=20) if post.url.endswith(('.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff'))]
        return images
    except Exception as e:
        st.write(f"Error fetching images: {e}")
        return []

# Define the Streamlit app
def run_app():
    st.title("Mental Health Classifier with Reddit Image Text Extraction and Translation")

    # Reddit username input
    username = st.text_input("Enter Reddit username to analyze images:")

    # Button to fetch images and classify
    if st.button("Analyze"):
        if username.strip() == "":
            st.write("Please enter a Reddit username.")
        else:
            # Fetch images
            image_urls = fetch_user_images(username)
            if not image_urls:
                st.write("No images found or unable to fetch images for this user.")
            else:
                predictions = []
                for image_url in image_urls:
                    st.write(f"Processing image from URL: {image_url}")
                    # Download and open the image
                    try:
                        response = requests.get(image_url)
                        image = Image.open(BytesIO(response.content))
                        st.image(image, caption="Fetched Image", use_column_width=True)

                        # Extract text from the image
                        extracted_text = reader.readtext(image, detail=0)
                        extracted_text = "\n".join(extracted_text)

                        st.subheader("Extracted Text")
                        st.text(extracted_text)

                        # Translate text to English if needed
                        try:
                            translated_text = GoogleTranslator(source='auto', target='en').translate(extracted_text)
                            st.subheader("Translated Text (to English)")
                            st.text(translated_text)
                        except Exception as e:
                            st.error(f"Translation failed: {e}")
                            return

                        # Vectorize the translated text and classify
                        input_vectorized = vectorizer.transform([translated_text])
                        prediction = model.predict(input_vectorized)
                        predictions.append(prediction[0])
                    except Exception as e:
                        st.write(f"Error processing image {image_url}: {e}")

                # Count the most common mental health issue
                if predictions:
                    issue_counts = Counter(predictions)
                    top_issue, top_count = issue_counts.most_common(1)[0]
                    top_percentage = (top_count / len(predictions)) * 100

                    # Output the result
                    st.write(f"The most frequently detected mental health concern is: {top_issue} appearing in {top_percentage:.2f}% of images.")

                    # Display all issues and their counts for reference
                    st.write("Mental health issue distribution across images:")
                    issue_distribution = pd.DataFrame(issue_counts.items(), columns=['Mental Health Issue', 'Count'])
                    st.write(issue_distribution)

# Run the app
if __name__ == '__main__':
    run_app()


Writing app09v2.py


In [ ]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("<NGROK_AUTH_TOKEN>") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app09v2.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://3141-35-204-74-166.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()